In [ ]:
import requests
import time
from datetime import datetime

# --- Configuration ---
# Using CoinGecko's free and simple API for the hackathon
# It's a good example of a "real-world signal"
API_URL = "https://api.coingecko.com/api/v3/simple/price?ids=ethereum&vs_currencies=usd%2Cinr"
KNOWLEDGE_FILE = "knowledge.metta"

def fetch_prices():
    """Fetches ETH prices in USD and INR from the CoinGecko API."""
    try:
        response = requests.get(API_URL)
        response.raise_for_status()  # Raises an exception for bad status codes
        data = response.json()
        prices = {
            "eth_usd": data["ethereum"]["usd"],
            "eth_inr": data["ethereum"]["inr"]
        }
        print(f"Successfully fetched prices: {prices}")
        return prices
    except requests.exceptions.RequestException as e:
        print(f"Error fetching prices from API: {e}")
        return None

def update_knowledge_base(prices):
    """
    Updates the knowledge.metta file with new facts.
    This uses MeTTa's S-expression syntax: (Predicate Subject Object)
    We are creating facts like: (= (price eth/usd) 3500.50)
    """
    if not prices:
        print("No prices to update.")
        return

    timestamp = datetime.utcnow().isoformat()

    # The 'w' mode overwrites the file, ensuring it always has the latest data.
    with open(KNOWLEDGE_FILE, "w") as f:
        # Asserting a fact for the price of ETH in USD
        fact_usd = f"(= (price eth/usd) {prices['eth_usd']})"
        f.write(fact_usd + "\n")
        
        # Asserting a fact for the price of ETH in INR
        fact_inr = f"(= (price eth/inr) {prices['eth_inr']})"
        f.write(fact_inr + "\n")
        
        # Asserting a fact for the update time, demonstrating provenance
        fact_time = f"(= (last-update) \"{timestamp}\")"
        f.write(fact_time + "\n")

    print(f"Knowledge base '{KNOWLEDGE_FILE}' updated successfully.")
    print("--- Contents ---")
    with open(KNOWLEDGE_FILE, "r") as f:
        print(f.read())
    print("----------------")


def run_oracle_agent():
    """Main loop for the oracle agent."""
    print("Starting Oracle Agent...")
    # For the hackathon, you can run this once to populate the file.
    # In a real system, it would run on a loop.
    # while True:
    #     prices = fetch_prices()
    #     update_knowledge_base(prices)
    #     print("Waiting for 60 seconds before next update...")
    #     time.sleep(60)
    prices = fetch_prices()
    update_knowledge_base(prices)


if __name__ == "__main__":
    # To run this file: `python oracle_agent.py`
    # It will create/update the `knowledge.metta` file in the same directory.
    run_oracle_agent()
